In [1]:
# ⚙️ 安裝 Faker（如未裝）
!pip install faker --quiet

import pandas as pd
import sqlite3
import random
from faker import Faker
from datetime import datetime, timedelta

# 初始化
fake = Faker()
random.seed(42)

# 💳 Exchange rate (to AUD)
exchange_rates = {
    "AUD": 1.0,
    "USD": 1.5,
    "HKD": 0.2
}

# 👤 1️⃣ 生成 cust_data（300位客戶）
num_customers = 300
tiers = ["GOLD", "SILVER", "VIP"]
start_date = datetime.now() - timedelta(days=365*10)

cust_list = []
for i in range(1, num_customers + 1):
    cin = f"C{i:04d}"
    name = fake.name()
    tier = random.choice(tiers)
    sin_dt = start_date + timedelta(days=random.randint(0, 365*10))
    cust_list.append([cin, name, tier, sin_dt.date()])

df_cust = pd.DataFrame(cust_list, columns=["CIN", "Cust Name", "Cust Tier", "Cust Sin Dt"])

# 💸 2️⃣ 生成 txn_data
txn_types = ["CASH", "WIRE", "ONLINE", "CARD", "ATM", "NON FACE TO FACE"]
currencies = ["HKD", "USD", "AUD"]
merchants = ["JPMorgan", "Amazon", "Coles", "Shell", "Apple Store", "PayPal", "Starbucks", "TransferWise"]

txn_list = []
for row in df_cust.itertuples():
    for _ in range(random.randint(1, 100)):
        txn_date = datetime.now() - timedelta(days=random.randint(0, 365*3))
        cur = random.choice(currencies)
        amount = round(random.uniform(50, 100000), 2)
        if random.random() < 0.02:
            amount *= 5  # 製造大額交易
        amount_aud = round(amount * exchange_rates[cur], 2)
        nf2f = random.choice(["Y", "N"])
        txn_type = random.choice(txn_types)
        counterparty = None if txn_type == "CASH" else random.choice(merchants)
        txn_list.append([
            txn_date.date(), row.CIN, cur, amount, amount_aud, nf2f, txn_type, counterparty
        ])

df_txn = pd.DataFrame(txn_list, columns=[
    "Txn Date", "CIN", "Txn Cur", "Txn Amt", "Txn Amt in AUD", "Nf2f", "Txn Type", "Counterparty name"
])

# 🧱 3️⃣ 建立 SQLite 資料庫（記憶體 or 本地）
conn = sqlite3.connect("bank_monitoring.db")
df_cust.to_sql("cust_data", conn, if_exists="replace", index=False)
df_txn.to_sql("txn_data", conn, if_exists="replace", index=False)

# 👁️‍🗨️ 預覽資料
print("👥 Customer Sample")
display(df_cust.head())

print("💳 Transaction Sample")
display(df_txn.head())

# 🔍 4️⃣ SQL 查詢邏輯：高風險行為

# 1. 大額現金交易
print("🔍 Large CASH Txn > 10,000")
query1 = """
SELECT * FROM txn_data
WHERE [Txn Type] = 'CASH' AND [Txn Amt in AUD] > 10000
LIMIT 10;
"""
display(pd.read_sql(query1, conn))

# 2. 大額電匯交易
print("🔍 Large WIRE Txn > 50,000")
query2 = """
SELECT * FROM txn_data
WHERE [Txn Type] = 'WIRE' AND [Txn Amt in AUD] > 50000
LIMIT 10;
"""
display(pd.read_sql(query2, conn))

# 3. 非面對面交易 + 大額
print("🔍 High Non Face-to-Face Txn > 10,000")
query3 = """
SELECT * FROM txn_data
WHERE Nf2f = 'Y' AND [Txn Amt in AUD] > 10000
LIMIT 10;
"""
display(pd.read_sql(query3, conn))

# 4. 新戶 + 大額交易
print("🔍 New Customer ≤ 30 days + Txn > 10,000")
query4 = """
SELECT T.*, C.[Cust Sin Dt]
FROM txn_data T
JOIN cust_data C ON T.CIN = C.CIN
WHERE julianday(T.[Txn Date]) - julianday(C.[Cust Sin Dt]) <= 30
AND T.[Txn Amt in AUD] > 10000
LIMIT 10;
"""
display(pd.read_sql(query4, conn))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.5 MB/s eta 0:00:00
👥 Customer Sample


,CIN,Cust Name,Cust Tier,Cust Sin Dt
0,C0001,Heather Jackson,VIP,2016-08-30
1,C0002,Andrew Cochran,GOLD,2023-09-24
2,C0003,Amanda Rodriguez,SILVER,2018-02-28
3,C0004,Sandra Hammond,GOLD,2016-12-23
4,C0005,Emma Rice,VIP,2016-07-24


💳 Transaction Sample


,Txn Date,CIN,Txn Cur,Txn Amt,Txn Amt in AUD,Nf2f,Txn Type,Counterparty name
0,2022-12-18,C0001,HKD,52016.04,10403.21,Y,ONLINE,Starbucks
1,2025-01-06,C0001,AUD,92081.05,92081.05,Y,NON FACE TO FACE,Apple Store
2,2022-07-26,C0001,USD,66705.95,100058.92,N,ATM,Coles
3,2024-05-02,C0001,USD,66509.70,99764.55,Y,ATM,Apple Store
4,2023-02-18,C0001,AUD,83380.83,83380.83,Y,CARD,PayPal


🔍 Large CASH Txn > 10,000


...

,Txn Date,CIN,Txn Cur,Txn Amt,Txn Amt in AUD,Nf2f,Txn Type,Counterparty name
0,2024-04-26,C0001,AUD,69650.61,69650.61,Y,WIRE,JPMorgan
1,2024-02-04,C0001,USD,33617.96,50426.94,Y,WIRE,Coles
2,2022-12-09,C0003,USD,76955.38,115433.07,N,WIRE,Shell
3,2023-05-16,C0003,AUD,80119.54,80119.54,Y,WIRE,TransferWise
4,2024-11-27,C0004,AUD,57434.63,57434.63,Y,WIRE,Starbucks
5,2022-09-25,C0004,USD,467928.75,701893.12,N,WIRE,TransferWise
6,2024-10-01,C0004,AUD,91617.23,91617.23,Y,WIRE,Coles
7,2022-06-05,C0004,USD,82344.88,123517.32,N,WIRE,Apple Store
8,2023-01-27,C0005,AUD,53022.68,53022.68,N,WIRE,PayPal
9,2023-11-27,C0005,USD,67383.51,101075.26,Y,WIRE,Shell


🔍 High Non Face-to-Face Txn > 10,000


,Txn Date,CIN,Txn Cur,Txn Amt,Txn Amt in AUD,Nf2f,Txn Type,Counterparty name
0,2022-12-18,C0001,HKD,52016.04,10403.21,Y,ONLINE,Starbucks
1,2025-01-06,C0001,AUD,92081.05,92081.05,Y,NON FACE TO FACE,Apple Store
2,2024-05-02,C0001,USD,66509.70,99764.55,Y,ATM,Apple Store
3,2023-02-18,C0001,AUD,83380.83,83380.83,Y,CARD,PayPal
4,2024-02-03,C0001,AUD,31076.43,31076.43,Y,CASH,None
5,2025-02-24,C0001,HKD,98394.41,19678.88,Y,CARD,Starbucks
6,2024-04-26,C0001,AUD,69650.61,69650.61,Y,WIRE,JPMorgan
7,2022-06-27,C0001,USD,62693.51,94040.26,Y,NON FACE TO FACE,Apple Store
8,2024-02-04,C0001,USD,33617.96,50426.94,Y,WIRE,Coles
9,2024-02-11,C0001,USD,69410.28,104115.42,Y,CARD,Starbucks


🔍 New Customer ≤ 30 days + Txn > 10,000


,Txn Date,CIN,Txn Cur,Txn Amt,Txn Amt in AUD,Nf2f,Txn Type,Counterparty name,Cust Sin Dt
0,2023-04-10,C0002,AUD,94032.17,94032.17,Y,CARD,JPMorgan,2023-09-24
1,2023-07-28,C0002,AUD,22175.65,22175.65,N,NON FACE TO FACE,Apple Store,2023-09-24
2,2023-05-08,C0002,AUD,90174.54,90174.54,Y,CARD,Apple Store,2023-09-24
3,2022-07-18,C0006,HKD,57292.77,11458.55,N,ATM,JPMorgan,2023-09-20
4,2022-12-08,C0006,AUD,21893.27,21893.27,Y,ONLINE,JPMorgan,2023-09-20
5,2022-11-20,C0006,USD,34890.27,52335.40,N,CASH,None,2023-09-20
6,2023-06-11,C0006,AUD,28144.45,28144.45,N,NON FACE TO FACE,Apple Store,2023-09-20
7,2023-10-15,C0006,AUD,26770.94,26770.94,Y,WIRE,Starbucks,2023-09-20
8,2023-08-09,C0006,HKD,76159.29,15231.86,N,WIRE,Apple Store,2023-09-20
9,2023-02-01,C0006,USD,45226.57,67839.85,Y,CARD,Apple Store,2023-09-20


In [2]:
print("🔍 Sum of CASH transactions per customer (in AUD)")

query = """
SELECT
  c.CIN,
  c.[Cust Tier],
  SUM(t.[Txn Amt in AUD]) AS txn_amt
FROM txn_data AS t
LEFT JOIN cust_data AS c ON t.CIN = c.CIN
WHERE t.[Txn Type] = 'CASH'
GROUP BY c.CIN, c.[Cust Tier]
ORDER BY txn_amt DESC
;
"""
# 執行 SQL 並顯示結果
df_result = pd.read_sql(query, conn)
display(df_result)

🔍 Sum of CASH transactions per customer (in AUD)


,CIN,Cust Tier,txn_amt
0,C0087,VIP,1409281.23
1,C0069,VIP,1370088.42
2,C0086,GOLD,1341503.70
3,C0270,VIP,1340623.14
4,C0265,GOLD,1292599.02
...,...,...,...
279,C0183,VIP,10924.38
280,C0138,SILVER,10529.08
281,C0299,VIP,9747.59
282,C0002,GOLD,8873.23


In [3]:
from google.colab import files
# Execute
df_result = pd.read_sql(query, conn)
display(df_result)

# Export to CSV and download
df_result.to_csv("CASH_Txn_Summary.csv", index=False)
files.download("CASH_Txn_Summary.csv")

,CIN,Cust Tier,txn_amt
0,C0087,VIP,1409281.23
1,C0069,VIP,1370088.42
2,C0086,GOLD,1341503.70
3,C0270,VIP,1340623.14
4,C0265,GOLD,1292599.02
...,...,...,...
279,C0183,VIP,10924.38
280,C0138,SILVER,10529.08
281,C0299,VIP,9747.59
282,C0002,GOLD,8873.23


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
query = """
SELECT
  c.CIN,
  c."Cust Tier",
  SUM(t."Txn Amt in AUD") AS txn_amt,
  COUNT(t."Txn Date") AS txn_cnt
FROM txn_data AS t
LEFT JOIN cust_data AS c ON t.CIN = c.CIN
WHERE t."Txn Type" = 'WIRE'
GROUP BY c.CIN, c."Cust Tier"
ORDER BY txn_amt DESC
LIMIT 10;
"""

df_result = pd.read_sql(query, conn)
display(df_result)

,CIN,Cust Tier,txn_amt,txn_cnt
0,C0153,VIP,1790121.45,16
1,C0106,VIP,1506631.40,20
2,C0265,GOLD,1385846.72,23
3,C0209,SILVER,1351035.77,22
4,C0072,SILVER,1258967.71,14
5,C0124,SILVER,1226987.90,14
6,C0054,SILVER,1223265.95,18
7,C0009,GOLD,1211738.33,17
8,C0022,SILVER,1190294.37,26
9,C0004,GOLD,1164431.81,12
